In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# importing the required libraries for the project
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
# cd /content/drive/MyDrive/AI/hotel_data

/content/drive/MyDrive/AI/hotel_data


In [5]:
# !unzip /content/drive/MyDrive/AI/hotel_data/Hotel_Reviews.csv.zip

Archive:  /content/drive/MyDrive/AI/hotel_data/Hotel_Reviews.csv.zip
  inflating: Hotel_Reviews.csv       


In [6]:
# uploading datasets
df = pd.read_csv('/content/drive/MyDrive/AI/hotel_data/Hotel_Reviews.csv')

In [7]:
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [8]:
df.shape

(515738, 17)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [10]:
print(df.Hotel_Address.head())

0     s Gravesandestraat 55 Oost 1092 AA Amsterdam ...
1     s Gravesandestraat 55 Oost 1092 AA Amsterdam ...
2     s Gravesandestraat 55 Oost 1092 AA Amsterdam ...
3     s Gravesandestraat 55 Oost 1092 AA Amsterdam ...
4     s Gravesandestraat 55 Oost 1092 AA Amsterdam ...
Name: Hotel_Address, dtype: object


In [12]:
# for simplicity change countries name to countries alpha-2 codes
df.Hotel_Address = df.Hotel_Address.str.replace('Netherlands', 'NL')
df.Hotel_Address = df.Hotel_Address.str.replace('United Kingdom', 'UK')
df.Hotel_Address = df.Hotel_Address.str.replace('France', 'FR')
df.Hotel_Address = df.Hotel_Address.str.replace('Spain', 'ES')
df.Hotel_Address = df.Hotel_Address.str.replace('Italy', 'IT')
df.Hotel_Address = df.Hotel_Address.str.replace('Austria', 'AT')

In [13]:
# splitting the hotel address and picking out the last string which would be countries
# and store them in new column name
df['countries'] = df.Hotel_Address.apply(lambda x: x.split(' ')[-1])

In [15]:
# checking on countries in new column
df.countries.unique()

array(['NL', 'UK', 'FR', 'ES', 'IT', 'AT'], dtype=object)

In [16]:
# checking out the necessary columns for our recommender
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'countries'],
      dtype='object')

In [18]:
# dropping unnecessay columns
df.drop(['Additional_Number_of_Scoring', 'Review_Date','Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng'], 1,inplace = True)

<ipython-input-18-6a3d777bb6a0>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Additional_Number_of_Scoring', 'Review_Date','Reviewer_Nationality',


In [19]:
# creating a function to convert the string of list into a normal list
# getting rid of the punctuations
def impute(column):
    column = column[0]
    if (type(column) != list):
        return "".join(literal_eval(column))
    else:
        return column

In [20]:
# check the tags column
df.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",NL
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",NL
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Family with young childre...",NL
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",NL
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",NL


In [21]:
# apply the impute function to the tags in the column
# get rid of the punctuations(commas and apostrophes)
df['Tags'] = df[['Tags']].apply(impute, axis = 1)
df.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,NL
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,Leisure trip Couple Duplex Double Room Sta...,NL
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,Leisure trip Family with young children Dup...,NL
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,Leisure trip Solo traveler Duplex Double Ro...,NL
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,Leisure trip Couple Suite Stayed 2 nights ...,NL


In [22]:
# for simplicity convert the tags and countries to lower case
df['Tags'] = df['Tags'].str.lower()
df['countries'] = df['countries'].str.lower()

In [28]:
# defining the recmmender fuction
def recommender(location, description):
    # dividing the text into small tokens (sentences into words)
    description = description.lower()
    word_tokenize(description)
    # applying stopwords
    stop_words = stopwords.words('english')
    # applying lemmataization
    lem = WordNetLemmatizer()
    # connecting packages and filtering
    filtered = {word for word in description if not word in stop_words}
    filtered_set = set()
    for fs in filtered:
        filtered_set.add(lem.lemmatize(fs))
# creating a variable that takes in the location and returns the following features
    country = df[df['countries'] == location.lower()]
    country = country.set_index(np.arange(country.shape[0]))
    list1 = []; list2 = []; cos = [];
    for i in range(country.shape[0]):
        temp_token = word_tokenize(country['Tags'][i])
        temp_set = [word for word in temp_token if not word in stop_words]
        temp2_set = set()
        for s in temp_set:
            temp2_set.add(lem.lemmatize(s))
        vector = temp2_set.intersection(filtered_set)
        cos.append(len(vector))
        # applying cosine similarity
    country['similarity'] = cos
    country = country.sort_values(by = 'similarity', ascending = False)
    country.drop_duplicates(subset = 'Hotel_Name', keep = 'first', inplace = True)
    country.sort_values('Average_Score', ascending = False, inplace = True)
    country.reset_index(inplace = True)
    return country[['Hotel_Name', 'Average_Score', 'Hotel_Address']].head(10)

In [29]:
# Enter your input
recommender('FR', 'business')

,Hotel_Name,Average_Score,Hotel_Address
0,Ritz Paris,9.8,15 Place Vend me 1st arr 75001 Paris FR
1,H tel de La Tamise Esprit de France,9.6,4 rue d Alger 1st arr 75001 Paris FR
2,Le Narcisse Blanc Spa,9.5,19 Boulevard De La Tour Maubourg 7th arr 75007...
3,Hotel The Peninsula Paris,9.5,19 avenue Kleber 16th arr 75116 Paris FR
4,Splendide Royal Paris,9.4,18 Rue du Cirque 8th arr 75008 Paris FR
5,Goralska R sidences H tel Paris Bastille,9.4,7 Boulevard Bourdon 4th arr 75004 Paris FR
6,Hotel Eiffel Blomet,9.4,78 Rue Blomet 15th arr 75015 Paris FR
7,Nolinski Paris,9.4,16 Avenue de l Opera 1st arr 75001 Paris FR
8,La Chambre du Marais,9.4,85 87 RUE DES ARCHIVES 3rd arr 75003 Paris FR
9,H tel D Aubusson,9.4,33 Rue Dauphine 6th arr 75006 Paris FR
